# Data - Cleaning

In this notebook we clean the data. This involves
- Taking the raw data which comes in a payment network format (i.e. sources correspond to sources of payment flows) and transform it into a production network formalism (where sources now correspond to sources of goods being produced)
- Removing the non-market and  industries with different input-output characteristics than national account data. This involves removing industries with the following SIC-2 codes: 45, 46, 47, 64, 65, 66 and 84 and above.
- Merge industries that would have zero aggregate output in at least one month with closely related industries.


## Importing Libraries

In [19]:
import sys
import os

# Data Manipulation
import pandas as pd
import pyreadr
import numpy as np

# Importing Data

The imported data is in the payment notation

In [89]:
### Fill in the correct data path
data_path_1 = '/Users/jlumma/Documents/...'
data_path_2 = '/Users/jlumma/Documents/...'
data_path_3 = '/Users/jlumma/Documents/...'


result_SIC5_V3_1 = pd.read_csv(data_path_1)
result_SIC5_V3_2 = pd.read_csv(data_path_2)
result_SIC5_V3_3 = pd.read_csv(data_path_3)


In [90]:
result_SIC5_V3 = pd.concat([result_SIC5_V3_1,result_SIC5_V3_2,result_SIC5_V3_3])

In [91]:
###### Renaming from and to columns while being in the payment formalism
###### 'from' corresponds to source of payment flow
###### 'to' corresponds to destination of payment flow
result_SIC5_V3 = result_SIC5_V3.rename(columns={'source_sic_revised': "from",'dest_sic_revised':'to'})

In [92]:
result_SIC5_V3['from'] = result_SIC5_V3['from'].astype(int)
result_SIC5_V3['to'] = result_SIC5_V3['to'].astype(int)

# Some rows have '*', we will remove these.
result_SIC5_V3 = result_SIC5_V3[result_SIC5_V3["jan_amt_15"] != '*']

In [72]:
#Let us remove the industries with leading '00'. At SIC-5 level this implies that 'from' and 'to' values 
#need to be larger than 999
result_SIC5_V3 = result_SIC5_V3[result_SIC5_V3['from']>999]
result_SIC5_V3 = result_SIC5_V3[result_SIC5_V3['to']>999]

In [94]:
result_SIC5_V3['from'] = result_SIC5_V3['from'].astype(str).str.zfill(5)
result_SIC5_V3['to'] = result_SIC5_V3['to'].astype(str).str.zfill(5)

In [74]:
result_SIC5_V3 = result_SIC5_V3.reset_index().drop(columns = 'index')

In [75]:

result_SIC5_V3 = result_SIC5_V3.groupby(['from', 'to']).sum().reset_index()

###### Keep Amount of Transactions

In [76]:
cols_amt = [c for c in result_SIC5_V3.columns if c[4:7] != 'cnt']
result_SIC5_V3_amt = result_SIC5_V3[cols_amt]

In [77]:
# Definition of the month and amount columns
month_columns = ['jan_amt_15', 'feb_amt_15', 'mar_amt_15', 'apr_amt_15',
       'may_amt_15', 'jun_amt_15', 'jul_amt_15', 'aug_amt_15', 'sep_amt_15',
       'oct_amt_15', 'nov_amt_15', 'dec_amt_15', 'jan_amt_16', 'feb_amt_16',
       'mar_amt_16', 'apr_amt_16', 'may_amt_16', 'jun_amt_16', 'jul_amt_16',
       'aug_amt_16', 'sep_amt_16', 'oct_amt_16', 'nov_amt_16', 'dec_amt_16',
       'jan_amt_17', 'feb_amt_17', 'mar_amt_17', 'apr_amt_17', 'may_amt_17',
       'jun_amt_17', 'jul_amt_17', 'aug_amt_17', 'sep_amt_17', 'oct_amt_17',
       'nov_amt_17', 'dec_amt_17', 'jan_amt_18', 'feb_amt_18', 'mar_amt_18',
       'apr_amt_18', 'may_amt_18', 'jun_amt_18', 'jul_amt_18', 'aug_amt_18',
       'sep_amt_18', 'oct_amt_18', 'nov_amt_18', 'dec_amt_18', 'jan_amt_19',
       'feb_amt_19', 'mar_amt_19', 'apr_amt_19', 'may_amt_19', 'jun_amt_19',
       'jul_amt_19', 'aug_amt_19', 'sep_amt_19', 'oct_amt_19', 'nov_amt_19',
       'dec_amt_19', 'jan_amt_20', 'feb_amt_20', 'mar_amt_20', 'apr_amt_20',
       'may_amt_20', 'jun_amt_20', 'jul_amt_20', 'aug_amt_20', 'sep_amt_20',
       'oct_amt_20', 'nov_amt_20', 'dec_amt_20', 'jan_amt_21', 'feb_amt_21',
       'mar_amt_21', 'apr_amt_21', 'may_amt_21', 'jun_amt_21', 'jul_amt_21',
       'aug_amt_21', 'sep_amt_21', 'oct_amt_21', 'nov_amt_21', 'dec_amt_21',
       'jan_amt_22', 'feb_amt_22', 'mar_amt_22', 'apr_amt_22', 'may_amt_22',
       'jun_amt_22', 'jul_amt_22']

In [78]:
result_SIC5_V3_amt.loc[:, month_columns] = result_SIC5_V3_amt[month_columns].astype(float)


In [79]:
result_SIC5_V3_amt = result_SIC5_V3_amt[(result_SIC5_V3_amt['from'] != value_to_remove) & (result_SIC5_V3_amt['to'] != value_to_remove)]


Let's relabel things to work in the production notation

In [80]:
result_SIC5_V3_amt['count'] = 1

In [81]:
##### Moving from the payment formalism to the production network formalism.
result_SIC5_V3_amt = result_SIC5_V3_amt.rename(columns={'from': 'to_pn', 'to': 'from_pn'})

## Removing non-market industries

We can alternatively also remove non-market industries as a way of cleaning the data. For that we remove Remove 45,46,47 (wholesale/retail), 64,65,66 (financial service industries), and everything above 84 onwards (non-market industries).

In [82]:
SIC2_non_market = result_SIC5_V3_amt.copy()

In [83]:
##### Truncating SIC-5 codes to the two-digit level for source and origin nodes
SIC2_non_market['to_pn_2'] = result_SIC5_V3_amt['to_pn'].astype(str).str[:2] 
SIC2_non_market['from_pn_2'] = result_SIC5_V3_amt['from_pn'].astype(str).str[:2]

In [84]:
SIC2_non_market['to_pn_2'] = SIC2_non_market['to_pn_2'].astype(float)
SIC2_non_market['from_pn_2'] = SIC2_non_market['from_pn_2'].astype(float)

In [85]:
####### Removing non-market and industries different from national account data
SIC5_non_market = SIC2_non_market[(SIC2_non_market['to_pn_2'] != 45) &(SIC2_non_market['from_pn_2'] != 45) &
               (SIC2_non_market['from_pn_2'] != 46) & (SIC2_non_market['to_pn_2'] != 46)&
               (SIC2_non_market['from_pn_2'] != 47) & (SIC2_non_market['to_pn_2'] != 47) &
               (SIC2_non_market['from_pn_2'] != 64) & (SIC2_non_market['to_pn_2'] != 64)&
                (SIC2_non_market['from_pn_2'] != 65) & (SIC2_non_market['to_pn_2'] != 65)&
                (SIC2_non_market['from_pn_2'] != 66) & (SIC2_non_market['to_pn_2'] != 66)&
                (SIC2_non_market['from_pn_2'] < 84) & (SIC2_non_market['to_pn_2'] < 84)
               ] 

######  Merging industries that will have zero output

This implies merging both source- and origin- industries

In [86]:
##### The case below is assuming we are merging industries such that aggregate output 
##### is non-zero across all industries. Here, we do it for the source industries.

SIC5_non_market.loc[:, 'from_pn'] = SIC5_non_market['from_pn'].replace({
    '25210': '25300',
    '33120': '33110',
    '33130': '33110',
    '33140': '33110',
    '33150': '33110',
    '33160': '33110',
    '33170': '33110',
    '33190': '33110',
    '77390': '77351',
    '74201': '74202',
    '82999': '82990',
    '01290': '01250',
    '01300': '01250',
    '01430': '01410',
    '01450': '01410',
    '01460': '01410',
    '01470': '01410',
    '01490': '01410',
    '01629': '01621',
    '01630': '01621',
    '01700': '01621',
    '10512': '10511',
    '10519': '10511',
    '10520': '10511',
    '13921': '13910',
    '13923': '13910',
    '13931': '13910',
    '13939': '13910',
    '20200': '20160',
    '23440': '23410',
    '23490': '23410',
    '24330': '24200'
})




In [87]:
##### The case below is assuming we are merging industries such that aggregate output 
##### is non-zero across all industries. Here, we do it for the origin industries.

SIC5_non_market.loc[:,'to_pn'] = SIC5_non_market['to_pn'].replace({
        '25210': '25300',
        '33120': '33110',
        '33130': '33110',
        '33140': '33110',
        '33150': '33110',
        '33160': '33110',
        '33170': '33110',
        '33190': '33110',
        '77390': '77351',
        '74201':'74202',
        '82999':'82990',
        '01290':'01250',
        '01300':'01250',
        '01430':'01410',
        '01450':'01410',
        '01460':'01410',
        '01470':'01410',
        '01490':'01410',
        '01629':'01621',
        '01630':'01621',
        '01700':'01621',
        '10512':'10511',
        '10519':'10511',
        '10520':'10511',
        '13921':'13910',
        '13923':'13910',
        '13931':'13910',
        '13939':'13910',
        '20200':'20160',
        '23440':'23410',
        '23490':'23410',
        '24330':'24200'
    })
    


### Saving Cleaned Data

In [38]:
# Assuming df is your DataFrame
subfolder = 'Saved_Data'
filename = 'Cleaned_Data_SIC5_non_market_removed.csv'
file_path = os.path.join(subfolder, filename)

SIC5_non_market.drop(columns = ['count','to_pn_2','from_pn_2']).to_csv(file_path, index=False)